In [8]:
import boto3
import botocore
import os, shutil

In [9]:
# deletes existing files in local destination folder
folder = '/Users/username/folder/'

for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

In [10]:
# Starts Boto3 and S3 sessions
session = boto3.session.Session(profile_name='myprofile')
s3_resource = session.resource('s3')

In [11]:
# Identifies the S3 bucket from where to download objects
my_bucket = s3_resource.Bucket('bucketname')

In [12]:
# lambda that gets the last modified time of objects
get_last_modified = lambda obj: int(obj.last_modified.strftime('%s'))

In [13]:
unsorted = []

# filters through the bucket and appends objects to the unsorted list
for file in my_bucket.objects.filter():
    unsorted.append(file)

# sorts unsorted by last modified time (14 most recent files)
latest_files = [obj.key for obj in sorted(unsorted, key=get_last_modified, reverse=True)][0:14]

In [14]:
# Downloads only the csv files from the bucket and places them in the local destination folder
for file in latest_files:
    if file.endswith('.csv'):
        s3_resource.meta.client.download_file('bucketname', file, '/Users/username/folder/' + file)